In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from util_kelly import simulate_kelly_strategy, StockMarketData, string_padding

# Betting on the S&P 500 using the Kelly Criterion

First load the data from your source csv file. The CSV should contain the stock price and a time stamp. Here we use the historical daily closing price for the S&P 500 index from 17th of February 1885 to 10th of february 2022. The data was downloaded from https://stooq.com/q/d/?s=^spx.

You can set the time interval that you want to analyse interactively.

In [3]:
DATE_MIN_BLOG = pd.Timestamp(1993, 1, 1).to_pydatetime()
DATE_MAX_BLOG = pd.Timestamp(2020, 12, 31).to_pydatetime()
PATH_TO_CSV = 'data/spx_day.csv'
simulation_data = StockMarketData(PATH_TO_CSV, parse_dates=['Date'], infer_datetime_format=True, index_col=0)

@widgets.interact(
    start_date=widgets.DatePicker(description='Start date', disabled=False, value=DATE_MIN_BLOG, continuous_update=False),
    end_date=widgets.DatePicker(description='End date', disabled=False, value=DATE_MAX_BLOG, continuous_update=False)
)
def load_data(start_date, end_date, simulation_data=widgets.fixed(simulation_data)):
    simulation_data.restrict_date(start_date, end_date)
    print(simulation_data.data)

interactive(children=(DatePicker(value=datetime.datetime(1993, 1, 1, 0, 0), description='Start date'), DatePic…

The investment strategy that is discussed here leverages fixed-period bets on the same asset, betting a variable fraction of the portfolio determined by the Kelly Criterion. The Kelly Fraction is defined as

$$ k = \frac{\mu - r}{sigma^2} $$

where $r$ is the risk-free rate, $\sigma$ is the standard deviation of returns, and $\mu$ is the mean return of the asset. In practice we estimate the mean and standard deviation of returns from a sample of historical returns.

$$ k_w = \frac{\mu_w - r}{\sigma_w^2} $$

and $w$ is the window size, that is the number of timesteps into the past used for the estimation of parameters $\mu_w$ and $\sigma_w$. For example

$$ \mu_w = \frac{1}{w} \sum_{i=1}^w return_{today - i}$$

Now we can start the interactive session. You can tune the following parameters:

- Window size : number of days that you want to look back to calculate the Kelly Factor
- Rebalancing interval: number of days that you want to wait before rebalancing your portfolio
- Minimum Kelly Factor: lower bound for the Kelly Factor
- Maximum Kelly Factor: upper bound for the Kelly Factor
- Kelly Fraction: Fraction of the Kelly Factor to use for investment into equity

and see how your strategy compares to simply buying and holding the S&P 500.

In [10]:
style = {'description_width': 'initial'}
layout=widgets.Layout(width='500px')
@widgets.interact(
    window=widgets.IntSlider(min=1, max=1008, step=10, value=252, description=string_padding('Window size', 35), style=style, layout=layout),
    rebalancing_interval=widgets.IntSlider(min=1, max=30, step=1, value=1, description=string_padding('Rebalancing Interval',30), style=style, layout=layout),
    min_kelly=widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, description=string_padding('Minimum Kelly Factor',29), style=style, layout=layout),
    max_kelly=widgets.FloatLogSlider(min=0, max=2, step=0.1, value=3, description=string_padding('Maximum Kelly Factor',28), style=style, layout=layout),
    kelly_fraction=widgets.FloatSlider(min=0, max=1, step=0.05, value=1, description=string_padding('Kelly Fraction',30), style=style, layout=layout)
)
def adjust_parameters(data=widgets.fixed(simulation_data), window=252, rebalancing_interval=1, min_kelly=0, max_kelly=3, kelly_fraction=1.0):

    k_cap = simulate_kelly_strategy(
        data.get_data(),
        rebalancing_interval=rebalancing_interval,
        kelly_fraction=kelly_fraction,
        window=window,
        min_kelly=min_kelly,
        max_kelly=max_kelly
    )

    fig, ax = plt.subplots(2, figsize=(18, 8), sharex=True)
    ax[0].plot(np.exp(k_cap['cum_returns']) * 100, label='Buy and Hold')
    ax[0].plot(np.exp(k_cap['strategy_cum_returns']) * 100, label='Kelly Model')
    ax[0].set_ylabel('Returns (%)')
    ax[0].set_title('Buy-and-hold vs. Rebalancing with Kelly Sizing')
    ax[0].legend()

    ax[1].plot(k_cap['kelly_fraction'])
    ax[1].set_ylabel('Leverage')
    ax[1].set_xlabel('Date')
    ax[1].set_title(f'Kelly Fraction (= Kelly Factor * {kelly_fraction})')

    plt.tight_layout()
    plt.show()


interactive(children=(IntSlider(value=252, description='Window size........................', layout=Layout(wi…

In the notebook '03_analysis_kelly_strategy.ipynb' I summarise my main findings.